# ETL: Renaming SPARCED 

1. Define Species Regex case
    a. if 1 param; 'k\D*\d*'
    b. if >1 param; 'k\D*\d*'+'_'+str(paramNum)

2. Make new_param_names using reactionId:
    a. param_name = "k"+str(reactionId)[1:]+'_'+str(param_Num)

3. rewrite formula with new name. 
    a. formula = formula with new name replacing old


In [1]:
import os
import re
import pandas as pd

ratelaws_df = pd.read_csv('../data/Ratelaws_old.tsv', sep='\t', header=0, index_col=0)

In [2]:
for reactionId, row in ratelaws_df.iterrows():
    j = 1
    paramids = [f'parameter{i}' for i in range(1, 9)]

    # Only get parameter names that are not NaN
    valid_params = [p for p in paramids if pd.notna(row[p])]
    
    formula = row['ratelaw']
    
    for _ in valid_params:

        if len(valid_params) == 1:
            param_regex = r'k\D*\d*'
        else:
            param_regex = r'k\D*\d*_' + str(j)
        
        new_param_name = "k" + str(reactionId)[1:] + '_' + str(j)

        compiled = re.compile(param_regex)
        matches = list(compiled.finditer(formula))
        
        for ematch in matches:
            formula = formula.replace(ematch.group(), new_param_name)

        j += 1

    ratelaws_df.at[reactionId, 'ratelaw'] = formula


In [3]:
ratelaws_df.to_csv('../data/Ratelaws_new.tsv', sep='\t', index=True)